In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
import math
import eigdec

In [ ]:
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette(["green"]))

In [ ]:
# Load image
img_raw = imread('src/shrek.jpeg')
img_raw.shape  # (height, width, color channels)

In [ ]:
img_sum = img_raw.sum(axis=2)  # From 3 color channels to 1 (greyscale)
img_bw = img_sum / img_sum.max()  # Normalize to [0, 1]
*img_bw.shape, int(img_bw.max())  # (height, width, color channels)

In [ ]:
plt.imshow(img_bw, cmap='gray')
plt.grid(False)

In [ ]:
img_mean = np.mean(img_bw, axis=0)
img_centered = img_bw - img_mean  # Center the data
covar_matrix = np.cov(img_centered, rowvar=False)  # Covariance matrix
covar_eigenvalues, covar_eigenvectors = eigdec.find_eigenvalues_and_vectors(covar_matrix)  # Eigenvectors - components

eigenvalues_var = (covar_eigenvalues / covar_eigenvalues.sum()) * 100  # Variance explained by each component
cumulative_var = np.cumsum(eigenvalues_var)  # Cumulative variance explained by components
opt_components_num = np.argmax(cumulative_var > 95)  # returns the 1st index with a value > 95

sns.lineplot(x=[i for i in range(len(covar_eigenvectors))],y=cumulative_var)
plt.scatter(opt_components_num, cumulative_var[opt_components_num], color='darkblue', s=75, zorder=3)
plt.axvline(opt_components_num, color='darkblue', linestyle='--', alpha=0.5)
plt.axhline(cumulative_var[opt_components_num], color='darkblue', linestyle='--', alpha=0.5)
plt.title("Explained variance by number of components")
plt.xlabel("Number of principal components")
plt.ylabel("Cumulative explained variance")

In [ ]:
opt_components_num  # number of components that explain 95% of the variance

In [ ]:
sorting_indices = np.argsort(covar_eigenvalues)[::-1]  # descending order
sorted_covar_eigenvectors = covar_eigenvectors[sorting_indices]
principal_components_opt = sorted_covar_eigenvectors[:, :opt_components_num]

reconstruct_img = lambda components: np.dot(np.dot(components.T, img_centered.T).T, components.T) + img_mean

In [ ]:
grid_size = math.ceil(math.sqrt(img_bw.shape[1] / opt_components_num))
fig = plt.figure(figsize=(15, 15))

components_num = opt_components_num
for i in range(grid_size):
    img_reconstructed = reconstruct_img(sorted_covar_eigenvectors[:, :components_num])
    plt.subplot(grid_size, grid_size, i + 1)
    plt.imshow(img_reconstructed, cmap='gray')
    plt.title(f"Number of components: {components_num}")
    plt.grid(False)

    components_num *= 2

plt.tight_layout()
plt.show()